# Fire emissions

We need fire emmissions for the plotting of net fluxes. This short notebook extracts and processes GFSASv12 fire emissions (stored in the g/data/ub8 folder) for use with AusEFlux plotting.

In [ ]:
import xarray as xr
import rioxarray as rxr
from odc.geo.xr import assign_crs
import pandas as pd
import numpy as np

In [ ]:
year_end = 2024

In [ ]:
years = [str(i) for i in range(2003,year_end+1)]

i=0
p=[]
for y in years:
    print(" {:02}/{:02}\r".format(i + 1, len(years)), end="")
    ds = xr.open_dataset('/g/data/ub8/au/FireEmissions/GFASv12.BurntCarbon.'+y+'.nc').BurntCarbon
    if y=='2024':
        ds['time'] = pd.date_range(start='1/1/2024', end='31/12/2024', freq='D') #time is wrong on this dataset!
    ds = ds.transpose('time', 'latitude', 'longitude') #coordinates are incorrectly ordered
    ds = assign_crs(ds, crs='EPSG:4326') # set the right projection
    gbox = ds.odc.geobox.to_crs('EPSG:3577')
    ds = ds.odc.reproject(how=gbox, resampling='bilinear')
    ds = ds * ds.odc.geobox.resolution.x**2 * 1e-15 #convert to a total carbon flux in petagrams
    ds= ds.resample(time='MS').sum() #resample daily values to monthly sums
    ds['time'] = ds['time'] + pd.Timedelta(14, 'd')
    # ds.attrs['units'] = 'PgC'
    ds=ds.rename('Fire Emissions (PgC)')
    p.append(ds)
    i+=1

ds = xr.concat(p, dim='time').sortby('time')
ds

In [ ]:
ds.mean('time').plot(robust=True)

In [ ]:
ds.to_netcdf('/g/data/xc0/project/AusEFlux/data/FireEmissions_10km_monthly.nc')